# 模块

其实我们前面已经多次引用过 Python 的各种非内置模块，也熟悉了两种引入模块中函数或变量的语法：
* `import xxxx`：引入 `xxxx` 模块，可以用 `xxxx.func()` 的语法调用其中的函数（变量也类似）；
* `from xxxx import func1, func2`：引入 `xxxx` 模块中的若干函数或变量，可以直接用不带模块名的 `func1` `func2` 来使用。

不过到目前为止我们都是使用现成模块（Python 自带或者我们通过 `pip install` 命令安装的第三方模块），我们自己也能创建自己的模块吗？

答案当然是肯定的。**模块**（*modules*），其实就是保存成文件的 Python 源代码。

当我们做点稍微复杂的事情时，就不太能够把所有代码写在一个文件里了，文件会变得巨大，非常难查找难阅读难修改。而 *module* 是 Python 提供给我们的模块化方法，让我们可以把源代码分开不同的文件，并在需要时引用其他文件里的代码。

这一章我们就介绍如何创建自己的模块以及如何使用。

## 创建模块

这几乎没啥可讲的，因为你只要把 Python 源代码存成一个 `.py` 的文件，这就是一个模块（*module*），而这个文件的文件名（不含后缀 `.py`）就是模块的名字。比如我们把我们之前写好的如下函数保存为一个叫做 `myutil.py` 的文件：

In [1]:
# 请在这里输入代码

from math import sqrt

def is_prime(n):
    """Return a boolean value based upon whether the argument n is a prime number."""
    if n < 2:
        return False
    
    if n in (2, 3):
        return True
    
    for i in range(2, int(sqrt(n))+1):
        if n % i == 0:
            return False
        
    return True

def say_hi(*names, greeting='Hi', capitalized=False):
    """
    Print a string, with a greeting to everyone.
    :param *names: tuple of names to be greeted.
    :param greeting: 'Hi' as default.
    :param capitalized: Whether name should be converted to capitalized before print.
    :return: None.
    """
    for name in names:
        if capitalized:
            name = name.capitalize()
        print(f"{greeting}, {name}!")

## 模块引入

保存好 `myutil.py` 文件之后，只要告诉解释器在哪里可以找到这个文件，就可以用 `import` 语句来引入和使用了，无论在 Jupyter Notebook 中还是其他 `.py` 文件中都是如此。

In [2]:
# 请在这里输入代码
import sys

sys.path

['E:\\lessons\\python\\get-start\\pilot-student',
 'D:\\program_files\\anaconda\\python37.zip',
 'D:\\program_files\\anaconda\\DLLs',
 'D:\\program_files\\anaconda\\lib',
 'D:\\program_files\\anaconda',
 '',
 'D:\\program_files\\anaconda\\lib\\site-packages',
 'D:\\program_files\\anaconda\\lib\\site-packages\\porespy-dev',
 'D:\\program_files\\anaconda\\lib\\site-packages\\openpnm-dev',
 'D:\\program_files\\anaconda\\lib\\site-packages\\locket-0.2.1-py3.7.egg',
 'D:\\program_files\\anaconda\\lib\\site-packages\\win32',
 'D:\\program_files\\anaconda\\lib\\site-packages\\win32\\lib',
 'D:\\program_files\\anaconda\\lib\\site-packages\\Pythonwin',
 'D:\\program_files\\anaconda\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\xuk11\\.ipython']

`sys` 是 Python 管理系统环境的模块，而 `sys.path` 保存着一个列表，里面是所有解释器“认识”的目录，这些目录都是解释器会自动去找模块的地方，我们可以看到，运行 Python 解释器的当前目录是自动包含在里面的；然后就是 Python 环境的库所在目录，我们用 `pip` 安装的库都会放在这些目录下。

注意：你的环境里显示出来的这个 `sys.path` 列表内容可能和我不一样，这没关系。

由于我把上面那个 `myutil.py` 放在这些 *notebooks* 所在目录下的 `assets` 子目录下，这个子目录不在上面显示的 `sys.path` 中，但我们可以把它加进去：

In [3]:
# 请在这里输入代码
sys.path.append('./assets/')

然后我们就可以方便的引入这个模块了，和我们使用 Python 自带的模块一样：

In [4]:
# 请在这里输入代码
import myutils

myutils.say_hi("Neo", "Trinity", "lyy")

Hi, Neo!
Hi, Trinity!
Hi, lyy!


In [5]:
# 请在这里输入代码
from myutils import is_prime

is_prime(47)

True

在上面的例子中，当使用 `import myutil` 的时候，就向当前环境引入了 `myutil` 文件中定义的所有函数，和下面的语句等价：

```python
from myutil import *
```

当然我们也可以只引入其中需要的函数，比如：

```python
from myutil import is_prime
```

这种情况下，就不需要使用 `myutil.is_prime()`，而是直接写 `is_prime()` 就好。

### 模块的分层

模块的使用可以分层。设想我们在一个目录下有很多 `.py` 源代码文件，我们可以分若干子目录，根据用途把 `.py` 文件分到这些子目录中去，比如我们有这么一个目录：

```
~/Code/some_project:
├── modules/
│   ├── __init__.py
│   ├── auth.py
│   ├── helper.py
│   ├── math/
│   │   ├── calculus.py
│   │   ├── logic.py
│   └── string/
│       ├── codec.py
│       └── l10n.py
├── setup.py
└── main.py
```

我们看到我们在这个目录下有一些主程序的源文件 `setup.py` `main.py`，然后一些工具性的源代码被放到了 `modules` 这个子目录下，并根据其用途分了一些更细的子目录（比如 `math` `string`）。这样的结构很合理和美观，也非常有伸缩性，以后再大的工程、再多源代码，也都能组织和管理好。那么要如何使用这些源代码呢？

第一步，你需要确保 `modules` 这个目录在解释器的搜索路径中，也就是将其加入 `sys.path` 列表（这件事可以在 `setup.py` 或者 `main.py` 里做）：

```python
sys.path.append('./modules/')
```

第二步，要确保 `modules` 目录下有一个叫做 `__init__.py` 的文件，这个文件通常为空文件，用于标识该目录是一个包含多个模块的包（*package*），所有这些模块都处在这个包的独立名字空间（*namespace*）下。

关于 *package* 和 *namespace* 目前还不是我们的重点，有兴趣可以参考[官方文档](https://docs.python.org/3/tutorial/modules.html#packages)。

然后就可以引入 `modules` 目录下的所有这些模块了，比如我们想引入 `auth.py` 里的所有函数，可以这样（假定是要在 `main.py` 里使用，下同）：

```python
import auth
```

如果要使用 `math` 目录下的 `calculus.py` 文件里的所有函数，可以：

```python
import math.calculus
```

也可以写成：

```python
from math import calculus
```

或者:

```python
from math.calculus import *
```

如果我们只要其中某几个函数，可以：

```python
from math.calculus import ode, pde
```

如果我们要一次引入 `string` 目录下的所有模块，可以：

```python
from string import *
```

所以，只要一个目录满足下面两个条件就是一个 Python 的软件包（*package*）：
1. 在 `sys.path` 列表中；
2. 里面有个 `__init__.py` 文件。

以这个目录作为根，下面所有 `.py` 源文件（包括无论多少级子目录下的）都可以作为 *module* 被其他 Python 程序引入和使用。引入的方法就是逐级目录加最后的文件名，中间用 `.` 隔开即可。

### 引入中的别名

引入模块或者模块中的函数时，可以设置模块或者函数的别名，这个能力很重要，因为难免我们引入的模块或者函数可能与我们正在写的东西重名，别名可以避免发生冲突。别名的语法很简单，在引入的时候加上 `as xxx` 就行，可以用于模块也可以用于函数，比如：

In [6]:
# 请在这里输入代码
from myutils import is_prime as isp

isp(59)

True

In [7]:
# 请在这里输入代码
import myutils as m

m.is_prime(57)

False

## 查看系统内置模块

前面用到的 `sys` 模块还有别的用途，可以用它提供的 `sys.builtin_module_names` 列表查看所有系统内置模块或者检查某个模块是不是内置模块：

In [8]:
# 请在这里输入代码
sys.builtin_module_names

('_abc',
 '_ast',
 '_bisect',
 '_blake2',
 '_codecs',
 '_codecs_cn',
 '_codecs_hk',
 '_codecs_iso2022',
 '_codecs_jp',
 '_codecs_kr',
 '_codecs_tw',
 '_collections',
 '_contextvars',
 '_csv',
 '_datetime',
 '_functools',
 '_heapq',
 '_imp',
 '_io',
 '_json',
 '_locale',
 '_lsprof',
 '_md5',
 '_multibytecodec',
 '_opcode',
 '_operator',
 '_pickle',
 '_random',
 '_sha1',
 '_sha256',
 '_sha3',
 '_sha512',
 '_signal',
 '_sre',
 '_stat',
 '_string',
 '_struct',
 '_symtable',
 '_thread',
 '_tracemalloc',
 '_warnings',
 '_weakref',
 '_winapi',
 'array',
 'atexit',
 'audioop',
 'binascii',
 'builtins',
 'cmath',
 'errno',
 'faulthandler',
 'gc',
 'itertools',
 'marshal',
 'math',
 'mmap',
 'msvcrt',
 'nt',
 'parser',
 'sys',
 'time',
 'winreg',
 'xxsubtype',
 'zipimport',
 'zlib')

In [11]:
print(sys.copyright)

Copyright (c) 2001-2019 Python Software Foundation.
All Rights Reserved.

Copyright (c) 2000 BeOpen.com.
All Rights Reserved.

Copyright (c) 1995-2001 Corporation for National Research Initiatives.
All Rights Reserved.

Copyright (c) 1991-1995 Stichting Mathematisch Centrum, Amsterdam.
All Rights Reserved.


和之前介绍的系统关键字列表一样，我们也最好不要用这里面有的名字，无论是作为我们自己的模块、类、函数还是变量名。

## 模块中不只有函数

一个模块文件中，不一定只包含函数；它也可以包含函数之外的可执行代码。`import` 这个命令不过是让解释器加载指定的模块（源代码），并且从头到尾执行一遍，其中定义函数的部分让这些函数可以被我们使用，其余部分就在 `import` 语句执行时执行一次而已。

有一个 Python 的彩蛋，恰好是可以用在此处的最佳例子——这个模块叫 this：

In [1]:
# 请在这里输入代码
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


这个模块的源代码我们也抄在下面，有兴趣可以试试能否独立读懂这个文件里的代码——对你来说，还挺练脑子的呢！

提示：这段代码先通过一个规则生成了一个密码表，保存在 `d` 这个字典中（也可以在我们学完字典这种数据容器之后再来读这段代码）；而后，将 `s` 这个变量中保存的 “密文” 翻译成了英文。或许，你可以试试，看看怎样能写个函数出来，给你一段英文，你可以把它加密成跟它一样的 “密文”？

In [4]:
# 请在这里输入代码
s = """Gur Mra bs Clguba, ol Gvz Crgref
Ornhgvshy vf orggre guna htyl.
Rkcyvpvg vf orggre guna vzcyvpvg.
Fvzcyr vf orggre guna pbzcyrk.
Pbzcyrk vf orggre guna pbzcyvpngrq.
Syng vf orggre guna arfgrq.
Fcnefr vf orggre guna qrafr.
Ernqnovyvgl pbhagf.
Fcrpvny pnfrf nera'g fcrpvny rabhtu gb oernx gur ehyrf.
Nygubhtu cenpgvpnyvgl orngf chevgl.
Reebef fubhyq arire cnff fvyragyl.
Hayrff rkcyvpvgyl fvyraprq.
Va gur snpr bs nzovthvgl, ershfr gur grzcgngvba gb thrff.
Gurer fubhyq or bar-- naq cersrenoyl bayl bar --boivbhf jnl gb qb vg.
Nygubhtu gung jnl znl abg or boivbhf ng svefg hayrff lbh'er Qhgpu.
Abj vf orggre guna arire.
Nygubhtu arire vf bsgra orggre guna *evtug* abj.
Vs gur vzcyrzragngvba vf uneq gb rkcynva, vg'f n onq vqrn.
Vs gur vzcyrzragngvba vf rnfl gb rkcynva, vg znl or n tbbq vqrn.
Anzrfcnprf ner bar ubaxvat terng vqrn -- yrg'f qb zber bs gubfr!"""

d = {}
for c in (65, 97):
    for i in range(26):
        d[chr(i+c)] = chr((i+13) % 26 + c)

print("".join([d.get(c, c) for c in s]))

The Zen of Python, by Tim Peters
Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


注意，虽然这个模块里什么函数都没有，但是 `import` 之后，里面所有全局变量我们都可以使用。

### 把英文加密成密文的函数。

In [12]:
ord('A') - ord('N')

-13

In [24]:
"-".join('abc*dec')

'a-b-c-*-d-e-c'

`string1.join(string2)`

以 `string1` 为连接符，将 `string2` 中的单个字符连接起来。

In [13]:
for c in (65, 97):
    for i in range(26):
        print(i+c, (i+13)%26+c)

65 78
66 79
67 80
68 81
69 82
70 83
71 84
72 85
73 86
74 87
75 88
76 89
77 90
78 65
79 66
80 67
81 68
82 69
83 70
84 71
85 72
86 73
87 74
88 75
89 76
90 77
97 110
98 111
99 112
100 113
101 114
102 115
103 116
104 117
105 118
106 119
107 120
108 121
109 122
110 97
111 98
112 99
113 100
114 101
115 102
116 103
117 104
118 105
119 106
120 107
121 108
122 109


In [26]:
for c in s:
    print (c)

G
u
r
 
M
r
a
 
b
s
 
C
l
g
u
b
a
,
 
o
l
 
G
v
z
 
C
r
g
r
e
f


O
r
n
h
g
v
s
h
y
 
v
f
 
o
r
g
g
r
e
 
g
u
n
a
 
h
t
y
l
.


R
k
c
y
v
p
v
g
 
v
f
 
o
r
g
g
r
e
 
g
u
n
a
 
v
z
c
y
v
p
v
g
.


F
v
z
c
y
r
 
v
f
 
o
r
g
g
r
e
 
g
u
n
a
 
p
b
z
c
y
r
k
.


P
b
z
c
y
r
k
 
v
f
 
o
r
g
g
r
e
 
g
u
n
a
 
p
b
z
c
y
v
p
n
g
r
q
.


S
y
n
g
 
v
f
 
o
r
g
g
r
e
 
g
u
n
a
 
a
r
f
g
r
q
.


F
c
n
e
f
r
 
v
f
 
o
r
g
g
r
e
 
g
u
n
a
 
q
r
a
f
r
.


E
r
n
q
n
o
v
y
v
g
l
 
p
b
h
a
g
f
.


F
c
r
p
v
n
y
 
p
n
f
r
f
 
n
e
r
a
'
g
 
f
c
r
p
v
n
y
 
r
a
b
h
t
u
 
g
b
 
o
e
r
n
x
 
g
u
r
 
e
h
y
r
f
.


N
y
g
u
b
h
t
u
 
c
e
n
p
g
v
p
n
y
v
g
l
 
o
r
n
g
f
 
c
h
e
v
g
l
.


R
e
e
b
e
f
 
f
u
b
h
y
q
 
a
r
i
r
e
 
c
n
f
f
 
f
v
y
r
a
g
y
l
.


H
a
y
r
f
f
 
r
k
c
y
v
p
v
g
y
l
 
f
v
y
r
a
p
r
q
.


V
a
 
g
u
r
 
s
n
p
r
 
b
s
 
n
z
o
v
t
h
v
g
l
,
 
e
r
s
h
f
r
 
g
u
r
 
g
r
z
c
g
n
g
v
b
a
 
g
b
 
t
h
r
f
f
.


G
u
r
e
r
 
f
u
b
h
y
q
 
o
r
 
b
a
r
-
-
 
n
a
q
 
c
e
r
s
r
e
n
o
y
l
 
b
a
y
l
 
b
a
r


In [32]:
d.get(' ', ' ')

' '

In [37]:
"".join([d.get(c, c) for c in s])

"The Zen of Python, by Tim Peters\nBeautiful is better than ugly.\nExplicit is better than implicit.\nSimple is better than complex.\nComplex is better than complicated.\nFlat is better than nested.\nSparse is better than dense.\nReadability counts.\nSpecial cases aren't special enough to break the rules.\nAlthough practicality beats purity.\nErrors should never pass silently.\nUnless explicitly silenced.\nIn the face of ambiguity, refuse the temptation to guess.\nThere should be one-- and preferably only one --obvious way to do it.\nAlthough that way may not be obvious at first unless you're Dutch.\nNow is better than never.\nAlthough never is often better than *right* now.\nIf the implementation is hard to explain, it's a bad idea.\nIf the implementation is easy to explain, it may be a good idea.\nNamespaces are one honking great idea -- let's do more of those!"

In [40]:
d

{'A': 'N',
 'B': 'O',
 'C': 'P',
 'D': 'Q',
 'E': 'R',
 'F': 'S',
 'G': 'T',
 'H': 'U',
 'I': 'V',
 'J': 'W',
 'K': 'X',
 'L': 'Y',
 'M': 'Z',
 'N': 'A',
 'O': 'B',
 'P': 'C',
 'Q': 'D',
 'R': 'E',
 'S': 'F',
 'T': 'G',
 'U': 'H',
 'V': 'I',
 'W': 'J',
 'X': 'K',
 'Y': 'L',
 'Z': 'M',
 'a': 'n',
 'b': 'o',
 'c': 'p',
 'd': 'q',
 'e': 'r',
 'f': 's',
 'g': 't',
 'h': 'u',
 'i': 'v',
 'j': 'w',
 'k': 'x',
 'l': 'y',
 'm': 'z',
 'n': 'a',
 'o': 'b',
 'p': 'c',
 'q': 'd',
 'r': 'e',
 's': 'f',
 't': 'g',
 'u': 'h',
 'v': 'i',
 'w': 'j',
 'x': 'k',
 'y': 'l',
 'z': 'm'}

In [45]:
e = {}
for c in (65, 97):
    for i in range(26):
        e[chr(i+c)] = chr((i+13) % 26 + c)
        
e

{'A': 'N',
 'B': 'O',
 'C': 'P',
 'D': 'Q',
 'E': 'R',
 'F': 'S',
 'G': 'T',
 'H': 'U',
 'I': 'V',
 'J': 'W',
 'K': 'X',
 'L': 'Y',
 'M': 'Z',
 'N': 'A',
 'O': 'B',
 'P': 'C',
 'Q': 'D',
 'R': 'E',
 'S': 'F',
 'T': 'G',
 'U': 'H',
 'V': 'I',
 'W': 'J',
 'X': 'K',
 'Y': 'L',
 'Z': 'M',
 'a': 'n',
 'b': 'o',
 'c': 'p',
 'd': 'q',
 'e': 'r',
 'f': 's',
 'g': 't',
 'h': 'u',
 'i': 'v',
 'j': 'w',
 'k': 'x',
 'l': 'y',
 'm': 'z',
 'n': 'a',
 'o': 'b',
 'p': 'c',
 'q': 'd',
 'r': 'e',
 's': 'f',
 't': 'g',
 'u': 'h',
 'v': 'i',
 'w': 'j',
 'x': 'k',
 'y': 'l',
 'z': 'm'}

d 与 e 是相等的。

In [46]:
msg = "".join([d.get(c, c) for c in s])
msg

"The Zen of Python, by Tim Peters\nBeautiful is better than ugly.\nExplicit is better than implicit.\nSimple is better than complex.\nComplex is better than complicated.\nFlat is better than nested.\nSparse is better than dense.\nReadability counts.\nSpecial cases aren't special enough to break the rules.\nAlthough practicality beats purity.\nErrors should never pass silently.\nUnless explicitly silenced.\nIn the face of ambiguity, refuse the temptation to guess.\nThere should be one-- and preferably only one --obvious way to do it.\nAlthough that way may not be obvious at first unless you're Dutch.\nNow is better than never.\nAlthough never is often better than *right* now.\nIf the implementation is hard to explain, it's a bad idea.\nIf the implementation is easy to explain, it may be a good idea.\nNamespaces are one honking great idea -- let's do more of those!"

In [44]:
encryp = "".join([e.get(c, c) for c in msg])
encryp

"Gur Mra bs Clguba, ol Gvz Crgref\nOrnhgvshy vf orggre guna htyl.\nRkcyvpvg vf orggre guna vzcyvpvg.\nFvzcyr vf orggre guna pbzcyrk.\nPbzcyrk vf orggre guna pbzcyvpngrq.\nSyng vf orggre guna arfgrq.\nFcnefr vf orggre guna qrafr.\nErnqnovyvgl pbhagf.\nFcrpvny pnfrf nera'g fcrpvny rabhtu gb oernx gur ehyrf.\nNygubhtu cenpgvpnyvgl orngf chevgl.\nReebef fubhyq arire cnff fvyragyl.\nHayrff rkcyvpvgyl fvyraprq.\nVa gur snpr bs nzovthvgl, ershfr gur grzcgngvba gb thrff.\nGurer fubhyq or bar-- naq cersrenoyl bayl bar --boivbhf jnl gb qb vg.\nNygubhtu gung jnl znl abg or boivbhf ng svefg hayrff lbh'er Qhgpu.\nAbj vf orggre guna arire.\nNygubhtu arire vf bsgra orggre guna *evtug* abj.\nVs gur vzcyrzragngvba vf uneq gb rkcynva, vg'f n onq vqrn.\nVs gur vzcyrzragngvba vf rnfl gb rkcynva, vg znl or n tbbq vqrn.\nAnzrfcnprf ner bar ubaxvat terng vqrn -- yrg'f qb zber bs gubfr!"

> 所以，这里用的加密和解密是同一套方法。即，一个字符通过这种方式执行两次能够得到它自己。

## `dir()` 函数

系统内置函数 `dir()` 可以查看一个已经引入的模块里有哪些可以访问的变量和函数：

In [51]:
# 请在这里输入代码
import sys

sys.path.append('./assets/')

import myutils

dir(myutils)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'is_prime',
 'say_hi',
 'sqrt']

In [52]:
mod_cont = dir(myutils)
mod_cont

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'is_prime',
 'say_hi',
 'sqrt']

In [68]:
for item in mod_cont:
    exec('myutils.'+item)

__builtins__
__cached__
__doc__
__file__
__loader__
__name__
__package__
__spec__
is_prime
say_hi
sqrt


In [93]:
myutils.__spec__

ModuleSpec(name='myutils', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000002514CDE6FD0>, origin='./assets\\myutils.py')

In [92]:
sys.__package__

''

In [91]:
myutils.__package__

''

In [90]:
myutils.__name__

'myutils'

In [89]:
myutils.__loader__

In [86]:
import sys

dir(sys)

['__breakpointhook__',
 '__displayhook__',
 '__doc__',
 '__excepthook__',
 '__interactivehook__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__stderr__',
 '__stdin__',
 '__stdout__',
 '_clear_type_cache',
 '_current_frames',
 '_debugmallocstats',
 '_enablelegacywindowsfsencoding',
 '_framework',
 '_getframe',
 '_git',
 '_home',
 '_xoptions',
 'api_version',
 'argv',
 'base_exec_prefix',
 'base_prefix',
 'breakpointhook',
 'builtin_module_names',
 'byteorder',
 'call_tracing',
 'callstats',
 'copyright',
 'displayhook',
 'dllhandle',
 'dont_write_bytecode',
 'exc_info',
 'excepthook',
 'exec_prefix',
 'executable',
 'exit',
 'flags',
 'float_info',
 'float_repr_style',
 'get_asyncgen_hooks',
 'get_coroutine_origin_tracking_depth',
 'get_coroutine_wrapper',
 'getallocatedblocks',
 'getcheckinterval',
 'getdefaultencoding',
 'getfilesystemencodeerrors',
 'getfilesystemencoding',
 'getprofile',
 'getrecursionlimit',
 'getrefcount',
 'getsizeof',
 'getswitchinterval',
 'gett

In [87]:
sys.__doc__

"This module provides access to some objects used or maintained by the\ninterpreter and to functions that interact strongly with the interpreter.\n\nDynamic objects:\n\nargv -- command line arguments; argv[0] is the script pathname if known\npath -- module search path; path[0] is the script directory, else ''\nmodules -- dictionary of loaded modules\n\ndisplayhook -- called to show results in an interactive session\nexcepthook -- called to handle any uncaught exception other than SystemExit\n  To customize printing in an interactive session or to install a custom\n  top-level exception handler, assign other functions to replace these.\n\nstdin -- standard input file object; used by input()\nstdout -- standard output file object; used by print()\nstderr -- standard error object; used for error messages\n  By assigning other file objects (or objects that behave like files)\n  to these, it is possible to redirect all of the interpreter's I/O.\n\nlast_type -- type of last uncaught exceptio

In [83]:
myutils.__file__

'./assets\\myutils.py'

In [88]:
print(myutils.__doc__)

None


In [79]:
myutils.__cached__

'./assets\\__pycache__\\myutils.cpython-37.pyc'

In [74]:
exec('myutils.'+item)

None


In [78]:
myutils.__builtins__

{'__name__': 'builtins',
 '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
 '__package__': '',
 '__loader__': _frozen_importlib.BuiltinImporter,
 '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>),
 '__build_class__': <function __build_class__>,
 '__import__': <function __import__>,
 'abs': <function abs(x, /)>,
 'all': <function all(iterable, /)>,
 'any': <function any(iterable, /)>,
 'ascii': <function ascii(obj, /)>,
 'bin': <function bin(number, /)>,
 'breakpoint': <function breakpoint>,
 'callable': <function callable(obj, /)>,
 'chr': <function chr(i, /)>,
 'compile': <function compile(source, filename, mode, flags=0, dont_inherit=False, optimize=-1)>,
 'delattr': <function delattr(obj, name, /)>,
 'dir': <function dir>,
 'divmod': <function divmod(x, y, /)>,
 'eval': <function eval(source, globals=None, locals=None, /)>,
 'exec': <function exec(s

可以看到前面有一大堆都不是我们写的，而是系统自动加进去的，这属于模块的**元数据**（*metadata*），你可以试试一个个看看里面是什么，比如：

In [0]:
# 请在这里输入代码

## `__main__` 模块

关于模块中不在任何函数里的部分，还有一点值得了解。

设想我们编写了一些函数，并在函数以外的部分写了一些代码来测试这些函数，然后将这些代码保存在一个模块（`.py` 文件）中供其他代码使用。如果别的代码 `import` 这个模块，那么那些测试代码就会被运行一遍，怎么看都觉得这不是个好事情，但是我们又不想删掉这些代码，因为我们以后对这个模块进行修改维护，还需要它们。Python 对此情况也提供了一个解决方案，这个方案就是使用特殊的模块名 `__main__`。

前面看到，当我们 `import` 一个模块之后，可以用 `myutil.__name__` 查看它的模块名，通常是它的文件名；Python 做了一个特殊的设定，那就是当我们在命令行界面用 `python myutil.py` 来运行这个文件时，它的模块名 `__name__` 的值为 `'__main__'`，这是个独一无二的值，不会出现在任何引入的模块中，这样我们就可以区分开“运行一个文件”和“引入一个模块”这两种行为，加以区别对待区别，也就是把模块写成这个样子：

```python
def func1();
    ...

def func2();
    ...

if __name__ == '__main__':
    # 把所有测试或者不希望被引入时执行的代码放在这里
```

这么做的结果是：

* 当 Python 文件被当作模块，被 import 语句导入时，if 判断失败，下面的代码不被执行；
* 当 Python 文件被 `python -m` 运行的时候，`if` 判断成功，下面的代码被执行。

## 小结

* 模块是 Python 管理多个源代码文件的基本单元，一个模块对应一个 `.py` 源文件；
* 模块名可以包含若干节，用 `.` 分开，`.` 分开的每一节对应子目录和文件名，比如当前目录下 `foo/goo/bar.py` 这个文件的模块名就是 `foo.goo.bar`；
* 可以用 `import` 语句来引入解释器能找到的任何模块，可以全模块引入也可以只引入指定的函数或者变量，还可以给引入的模块、函数或变量用 `as` 指定别名；
* 解释器会在 `sys.path` 这个变量保存的所有目录下查找我们想引入的模块；
* 了解 `sys.builtin_module_names` 和 `dir()` 的用法，有助于我们探索当前环境的可用模块以及了解模块的信息；
* 可以把不希望被引入时执行的代码放在 `if __name__ == '__main__':` 判断下面，让一个文件既可以被当做模块引入，也可以作为一个程序独立执行。